In [16]:
#importing required packages
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim
from torchvision import datasets, transforms
import os
import numpy as np
import pandas as pd


In [14]:
#Hyperparamaters
batch_size = 100
sequence_len= 28
input_len = 28
num_layers = 2
num_class = 10
hidden_size = 128
epochs = 5
learning_rate = 0.01

In [43]:
#preparing datasets
train_data = datasets.FashionMNIST(root= './datasets/fashion_mnist', train=True, transform=transforms.ToTensor(), download=True)
test_data = datasets.FashionMNIST(root= "./datasets/fashion_mnist", train=False, transform=transforms.ToTensor())

train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)



In [57]:
#Creating a model
class LSTM(nn.Module):
    def __init__(self, input_len, hidden_size, num_class, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_len, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, num_class)
    
    def forward(self, X):
        hidden_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
        cell_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
        out,_ = self.lstm(X, (hidden_states, cell_states))
        output = self.output_layer(out[:, -1,:])
        return output

In [58]:
#model instance
model = LSTM(input_len, hidden_size, num_class, num_layers)
print(model)

LSTM(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=10, bias=True)
)


In [59]:
#creating a loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = learning_rate)
#creating second optimizer for better model performance
optimizer2 = optim.Adam(model.parameters(), lr= learning_rate)

In [60]:
#training the LSTM model
def training(epochs, model, train_dataloader, loss_function):
    total_steps = len(train_dataloader)
    
    for epoch in range(epochs):
        for batch, (image, labels) in enumerate(train_dataloader):
            images = image.reshape(-1, sequence_len, input_len)
            
            output = model(images)
            loss = loss_function(output, labels)

            optimizer2.zero_grad()
            loss.backward()
            optimizer2.step()

            if(batch+1)%100 == 0:
                print(f"Epoch count: {epoch}, Batch: {batch+1}, Loss: {loss.item()}")

In [53]:
#starting the training loop
training(epochs, model, train_dataloader, loss_function )

Epoch count: 0, Batch: 100, Loss: 2.301945447921753
Epoch count: 0, Batch: 200, Loss: 2.299060583114624
Epoch count: 0, Batch: 300, Loss: 2.29500675201416
Epoch count: 0, Batch: 400, Loss: 2.2909274101257324
Epoch count: 0, Batch: 500, Loss: 2.2930355072021484
Epoch count: 0, Batch: 600, Loss: 2.290353298187256
Epoch count: 1, Batch: 100, Loss: 2.2860372066497803
Epoch count: 1, Batch: 200, Loss: 2.285726547241211
Epoch count: 1, Batch: 300, Loss: 2.276427984237671
Epoch count: 1, Batch: 400, Loss: 2.2731900215148926
Epoch count: 1, Batch: 500, Loss: 2.274792432785034
Epoch count: 1, Batch: 600, Loss: 2.266252279281616
Epoch count: 2, Batch: 100, Loss: 2.256728172302246
Epoch count: 2, Batch: 200, Loss: 2.2575173377990723
Epoch count: 2, Batch: 300, Loss: 2.235334873199463
Epoch count: 2, Batch: 400, Loss: 2.2238519191741943
Epoch count: 2, Batch: 500, Loss: 2.2220988273620605
Epoch count: 2, Batch: 600, Loss: 2.204092025756836
Epoch count: 3, Batch: 100, Loss: 2.168156623840332
Epoch 

In [61]:
#with Adam optimizer
training(epochs, model, train_dataloader, loss_function )

Epoch count: 0, Batch: 100, Loss: 1.0342955589294434
Epoch count: 0, Batch: 200, Loss: 0.7790061235427856
Epoch count: 0, Batch: 300, Loss: 0.5219201445579529
Epoch count: 0, Batch: 400, Loss: 0.5313878059387207
Epoch count: 0, Batch: 500, Loss: 0.6365305185317993
Epoch count: 0, Batch: 600, Loss: 0.3915071189403534
Epoch count: 1, Batch: 100, Loss: 0.3867504894733429
Epoch count: 1, Batch: 200, Loss: 0.3667123317718506
Epoch count: 1, Batch: 300, Loss: 0.3222603499889374
Epoch count: 1, Batch: 400, Loss: 0.42798274755477905
Epoch count: 1, Batch: 500, Loss: 0.46218881011009216
Epoch count: 1, Batch: 600, Loss: 0.2788180708885193
Epoch count: 2, Batch: 100, Loss: 0.3336666226387024
Epoch count: 2, Batch: 200, Loss: 0.3066610097885132
Epoch count: 2, Batch: 300, Loss: 0.35651057958602905
Epoch count: 2, Batch: 400, Loss: 0.3164610266685486
Epoch count: 2, Batch: 500, Loss: 0.45928364992141724
Epoch count: 2, Batch: 600, Loss: 0.23383377492427826
Epoch count: 3, Batch: 100, Loss: 0.30939

In [63]:
#Testing the model
test_images, test_labels = next(iter(test_dataloader))
test_labels

tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 2, 5, 7, 9,
        1, 4, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 6, 1, 3, 7, 6, 7, 2, 1,
        2, 2, 4, 4, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5, 1, 1, 2, 3, 9, 8, 7, 0,
        2, 6, 2, 3, 1, 2, 8, 4, 1, 8, 5, 9, 5, 0, 3, 2, 0, 6, 5, 3, 6, 7, 1, 8,
        0, 1, 4, 2])

In [64]:
test_output =  model(test_images.view(-1,28,28))

In [65]:
predict = torch.max(test_output, 1)[1]
predict

tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 8, 3, 4, 1, 2, 2, 8, 0, 2, 5, 7, 5,
        1, 2, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 0, 1, 6, 9, 6, 7, 2, 1,
        2, 2, 4, 4, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5, 1, 1, 4, 3, 7, 8, 7, 0,
        2, 6, 2, 3, 1, 2, 8, 4, 1, 8, 5, 9, 5, 0, 3, 2, 0, 2, 5, 3, 6, 7, 1, 8,
        0, 1, 4, 2])

In [69]:
correct_prediction = sum([1 for i in range(100) if predict[i] == test_labels[i]])/100
print(f'Correct prediction = {correct_prediction *100}%')

Correct prediction = 90.0%


In [79]:
#checking how many test dataloader items were correctly classified 
def test_func(dataloader, model, loss_function, optimizer2):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss , correct = 0,0

    with torch.no_grad():
        for X, y in dataloader:
            X= X.reshape(-1,28,28)
            pred = model(X)
            test_loss += loss_function(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Accuracy: {(100*correct)}, Average loss: {test_loss}")
    return 100*correct

test_func(test_dataloader,model, loss_function, optimizer2)

Accuracy: 88.34, Average loss: 0.3203294937312603


88.34